# LightGlue-ONNX Demo
This demo shows how to export LightGlue to ONNX and perform inference with ONNXRuntime.

In [ ]:
from export import export_onnx
from infer import infer
import time

## ONNX Export
In this example we use SuperPoint features combined with LightGlue.

In [ ]:
extractor_type = "superpoint"  # "disk"
extractor_path = f"weights/{extractor_type}.onnx"
lightglue_path = f"weights/{extractor_type}_lightglue.onnx"

export_onnx(
    extractor_type=extractor_type,
    extractor_path=extractor_path,
    lightglue_path=lightglue_path,
    dynamic=True,  # for dynamic image size
    max_num_keypoints=None,
)

## ONNX Inference

In [ ]:
m_kpts0, m_kpts1 = infer(
    img_paths=["assets/tilt_image.png", "assets/tile_image.png"],
    extractor_type=extractor_type,
    extractor_path=extractor_path,
    lightglue_path=lightglue_path,
    img_size=512,
    viz=True,
    trt=False,
)

### ALTERNATIVE

In [ ]:
from lightglue import LightGlue, SuperPoint
from lightglue.utils import load_image, read_image, numpy_image_to_torch, rbd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time

device='cuda'

# SuperPoint+LightGlue
extractor = SuperPoint(max_keypoints=3840, detection_threshold=0.0005).eval().to(device)  # load the extractor
matcher = LightGlue(features='superpoint', depth_confidence=0.99, width_confidence=0.99, filter_threshold=0.001).eval().to(device)  # load the matcher

img0 = read_image('assets/sacre_coeur1.jpg')
img1 = read_image('assets/sacre_coeur2.jpg')


start_clock = time.time()

# extract local features
image0 = numpy_image_to_torch(img0).to(device)
feats0 = extractor.extract(image0)

end_clock = time.time()
time_diff = end_clock - start_clock
print(f"Execution Time: {time_diff:.4f} seconds")

start_clock = time.time()

# extract local features
image1 = numpy_image_to_torch(img1).to(device)
feats1 = extractor.extract(image1)

end_clock = time.time()
time_diff = end_clock - start_clock
print(f"Execution Time: {time_diff:.4f} seconds")



# match the features
start_clock = time.time()
matches01 = matcher({'image0': feats0, 'image1': feats1})
feats0, feats1, matches01 = [rbd(x) for x in [feats0, feats1, matches01]]  # remove batch dimension


matches = matches01['matches']  # indices with shape (K,2)
print(f"Number of matches: {matches.shape[0]}")
points0 = feats0['keypoints'][matches[..., 0]]  # coordinates in image #0, shape (K,2)
points1 = feats1['keypoints'][matches[..., 1]]  # coordinates in image #1, shape (K,2)

end_clock = time.time()
time_diff = end_clock - start_clock
print(f"Execution Time: {time_diff:.4f} seconds")
        
# Convert points to OpenCV KeyPoints
img0_kpts = [cv2.KeyPoint(p[0], p[1], 1) for p in feats0['keypoints'].cpu().numpy()]
img1_kpts = [cv2.KeyPoint(p[0], p[1], 1) for p in feats1['keypoints'].cpu().numpy()]

# Convert matches
good_matches = [cv2.DMatch(int(m[0]), int(m[1]), 0) for m in matches]

# Draw the matches
matched_image = cv2.drawMatches(
    img0, img0_kpts,  # Image 1 and its keypoints
    img1, img1_kpts,  # Image 2 and its keypoints
    good_matches,  # Matches between keypoints
    None,  # Output image, pass None to create automatically
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS  # Draw only matched points
)

plt.figure(figsize=(20, 20))
plt.imshow(matched_image)
plt.axis('off')
plt.show()

cv2.imwrite('assets/matched_image_coracao.png', cv2.cvtColor(matched_image, cv2.COLOR_RGB2BGR))


### ALTERNATIVE ONNX

In [ ]:
import numpy as np
import cv2
import time
import onnxruntime as ort
from lightglue_dynamo import viz
from lightglue_dynamo.preprocessors import SuperPointPreprocessor

model_path = "weights/superpoint_lightglue_pipeline.fp16.onnx"
output_path = "matches_onnx.png"
width = 1024
height = 1024
fp16 = True
profile = True
device = "cuda"

#raw_images = [cv2.resize(cv2.imread(str(i)), (width, height)) for i in raw_images]

img1 = cv2.imread('assets/sacre_coeur1.jpg')
img2 = cv2.imread('assets/sacre_coeur2.jpg')

img1 = cv2.resize(img1, (width, height))
img2 = cv2.resize(img2, (width, height))
raw_images = [img1, img2]
images = np.stack(raw_images)
images = SuperPointPreprocessor.preprocess(images)
images = images.astype(np.float16 if fp16 and device != "tensorrt" else np.float32)
session_options = ort.SessionOptions()
#session_options.enable_profiling = profile
providers = [("CPUExecutionProvider", {})]
if device == "cuda":
    providers.insert(0, ("CUDAExecutionProvider", {}))
elif device == "tensorrt":
    providers.insert(0, ("CUDAExecutionProvider", {}))
    providers.insert(
        0,
        (
            "TensorrtExecutionProvider",
            {
                "trt_engine_cache_enable": True,
                "trt_engine_cache_path": "weights/.trtcache_engines",
                "trt_timing_cache_enable": True,
                "trt_timing_cache_path": "weights/.trtcache_timings",
                "trt_fp16_enable": fp16,
            },
        ),
    )
elif device == "openvino":
    providers.insert(0, ("OpenVINOExecutionProvider", {}))
session = ort.InferenceSession(model_path, session_options, providers)
for _ in range(4 if profile else 1):
    
    try:
        start_clock = time.time()
        
        keypoints, matches, mscores = session.run(None, {"images": images})
        matched_points0 = keypoints[0][matches[..., 1]]
        matched_points1 = keypoints[1][matches[..., 2]]
        end_clock = time.time()
        time_diff = end_clock - start_clock
        print(f"Execution Time: {time_diff:.4f} seconds")
    #exception oom error  
    except MemoryError as e:
        print(f'OOM error encountered: {e}')
        continue

viz.plot_images(raw_images)
viz.plot_matches(matched_points0, matched_points1, color="lime", lw=0.2)
if output_path is None:
    viz.plt.show()
else:
    viz.save_plot(output_path)

### ALTERNATIVE POLYGRAPHY

In [ ]:
import numpy as np
import cv2
import typer
from polygraphy.backend.common import BytesFromPath
from polygraphy.backend.trt import (
    EngineFromBytes,
    TrtRunner,
)
from lightglue_dynamo import viz
from lightglue_dynamo.preprocessors import SuperPointPreprocessor

model_path = "weights/superpoint_lightglue_pipeline.fp16.engine"
output_path = "matches_polygraphy.png"
width = 1024
height = 1024
fp16 = True
profile = True

img1 = cv2.imread('assets/sacre_coeur1.jpg')
img2 = cv2.imread('assets/sacre_coeur2.jpg')
img1 = cv2.resize(img1, (width, height))
img2 = cv2.resize(img2, (width, height))
raw_images = [img1, img2]
images = np.stack(raw_images)
images = SuperPointPreprocessor.preprocess(images)
images = images.astype(np.float16 if fp16 else np.float32)
# Build TensorRT engine
build_engine = EngineFromBytes(BytesFromPath(str(model_path)))
runner = TrtRunner(build_engine)
runner.activate()
        
for _ in range(10 if profile else 1):  # Warm-up if profiling
    outputs = runner.infer(feed_dict={"images": images})
    keypoints, matches, mscores = outputs["keypoints"], outputs["matches"], outputs["mscores"]  # noqa: F841
    print(f"Matches size: {matches.shape}")
        
    if profile:
        typer.echo(f"Inference Time: {runner.last_inference_time():.3f} s")
        
viz.plot_images(raw_images)
viz.plot_matches(keypoints[0][matches[..., 1]], keypoints[1][matches[..., 2]], color="lime", lw=0.2)
if output_path is None:
    viz.plt.show()
else:
    viz.save_plot(output_path)

In [ ]:
import torch
import time
from torch.func import jvp, jacfwd

# Example: residual function r(x)
def residual_func(x):
    return torch.sin(x.sum()) + torch.arange(10000.0, device=x.device) * x.sum()

# Compute J^T J efficiently without storing J
def compute_jtj_no_storage(residual_func, params):
    N = params.numel()  # Number of parameters
    JTJ = torch.zeros((N, N), dtype=params.dtype, device=params.device)  # Initialize J^T J

    # Standard basis vectors
    identity = torch.eye(N, dtype=params.dtype, device=params.device)
    for i in range(N):
        e_i = identity[i]
        e_i[i] = 1.0

        # Compute J @ e_i (i-th column of J)
        _, jvp_result = jvp(residual_func, (params,), (e_i,))
        
        # Accumulate J^T J for the upper triangular part
        JTJ[i, i:] += (jvp_result.unsqueeze(0) @ jvp_result.unsqueeze(1))[0, :N - i]

    # Copy the upper triangular part to the lower triangular part
    i_lower = torch.tril_indices(N, N, -1)
    JTJ[i_lower[0], i_lower[1]] = JTJ.T[i_lower[0], i_lower[1]]

    return JTJ

# Compare performance and memory usage
def compare_methods(residual_func, params):
    # Measure no-storage method
    start_time = time.perf_counter()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    JTJ_no_storage = compute_jtj_no_storage(residual_func, params)
    elapsed_no_storage = time.perf_counter() - start_time

    # Measure full Jacobian method
    start_time = time.perf_counter()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    J = jacfwd(residual_func, argnums=0)(params)
    JTJ_full = J.T @ J
    elapsed_full = time.perf_counter() - start_time

    # Check memory usage (GPU only)
    if torch.cuda.is_available():
        memory_no_storage = torch.cuda.memory_allocated()
        torch.cuda.empty_cache()
        memory_full = torch.cuda.memory_allocated()
        return elapsed_no_storage, elapsed_full, memory_no_storage, memory_full
    else:
        return elapsed_no_storage, elapsed_full

# Example usage
params = torch.randn(500, requires_grad=True)  # 500 parameters (N=500)
results = compare_methods(residual_func, params)

# Display results
if len(results) == 4:  # GPU with memory tracking
    elapsed_no_storage, elapsed_full, memory_no_storage, memory_full = results
    print(f"No-storage method: {elapsed_no_storage:.4f}s, {memory_no_storage / 1e6:.2f} MB")
    print(f"Full Jacobian method: {elapsed_full:.4f}s, {memory_full / 1e6:.2f} MB")
else:  # CPU only
    elapsed_no_storage, elapsed_full = results
    print(f"No-storage method: {elapsed_no_storage:.4f}s")
    print(f"Full Jacobian method: {elapsed_full:.4f}s")